# IDP Backend Server Setup

This notebook runs the FastAPI backend server with ngrok tunnel.

In [ ]:
# Install required packages
!pip install -q fastapi==0.110.0 \
    uvicorn[standard]==0.27.1 \
    python-multipart==0.0.9 \
    torch==2.2.1+cpu \
    transformers==4.38.2 \
    Pillow==10.2.0 \
    paddleocr==2.7.0.3 \
    python-magic-bin==0.4.14 \
    spacy==3.7.4 \
    pyngrok==7.1.5 \
    python-dotenv==1.0.1

In [ ]:
# Download spaCy model
!python -m spacy download en_core_web_sm

In [ ]:
# Create necessary directories
!mkdir -p models/layout models/spacy data/annotations

In [ ]:
%%writefile main.py
from fastapi import FastAPI, File, UploadFile, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from typing import Dict, Any
import uvicorn
from PIL import Image
import io
import magic
import os
from models.document_processor import DocumentProcessor

app = FastAPI()

# Configure CORS - allow all origins in development
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Initialize processor
doc_processor = DocumentProcessor()

ALLOWED_MIME_TYPES = [
    'image/jpeg',
    'image/png',
    'image/tiff',
    'application/pdf'
]

@app.post("/process-document")
async def process_document(file: UploadFile = File(...)) -> Dict[str, Any]:
    try:
        content = await file.read()
        mime = magic.Magic(mime=True)
        file_type = mime.from_buffer(content)
        
        if file_type not in ALLOWED_MIME_TYPES:
            raise HTTPException(status_code=400, detail=f"Unsupported file type: {file_type}")
        
        image = Image.open(io.BytesIO(content))
        result = doc_processor.process_image(image)
        return result
        
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/health")
async def health_check():
    return {"status": "healthy"}

In [ ]:
# Configure and start ngrok
from pyngrok import ngrok

# Get your token from https://dashboard.ngrok.com/get-started/your-authtoken
NGROK_AUTH_TOKEN = "your_ngrok_auth_token"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Create tunnel
public_url = ngrok.connect(8000)
print(f'\nPublic URL: {public_url}\n')
print('Copy this URL and update your frontend .env file with:')
print(f'VITE_API_URL={public_url}')

In [ ]:
# Start the FastAPI server
import nest_asyncio
import uvicorn

nest_asyncio.apply()
uvicorn.run('main:app', host='0.0.0.0', port=8000)